In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 64

num_epochs = 10

X_train = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\X_train.npy")
y_train = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\y_train.npy")
X_val   = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\X_val.npy")
y_val   = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\y_val.npy")
X_test  = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\X_test.npy")
y_test  = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\y_test.npy")

N = len(X_train)
feature_dim = X_train.shape[2]
num_labels = y_train.shape[1]
# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_t   = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t   = torch.tensor(y_val, dtype=torch.float32).to(device)
X_test_t  = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).to(device)

train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t, y_val_t)
test_ds  = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

class CNNTransformer(nn.Module):
    """
    x shape attendu : (batch, seq_len, feature_dim)
    """
    def __init__(self, feature_dim, num_classes, d_model=128, nhead=4, num_layers=4):
        super().__init__()
        # ... (CNN, Transformer et Classification comme dans votre code original) ...
        self.conv = nn.Sequential(
            nn.Conv1d(feature_dim, d_model, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(d_model, d_model, kernel_size=3, padding=1),
            nn.ReLU()
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.cls_head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes)
        )

    def forward(self, x):
        # x : (B, L, F) -> conv attend (B, F, L)
        x = x.transpose(1, 2)
        x = self.conv(x)          # (B, d_model, L)
        x = x.transpose(1, 2)     # retour (B, L, d_model)
        x = self.transformer(x)   # encoder
        x = x[:, -1, :]           # dernier token
        return self.cls_head(x) # (B, num_labels)


# -----------------------------
# Instanciation
# -----------------------------
model = CNNTransformer(
    feature_dim=feature_dim, # Utilise la dimension extraite
    num_classes=num_labels, # Utilise le nombre de labels (6)
    d_model=128,
    nhead=4,
    num_layers=4
).to(device)


# --- Fonction d'entraînement/validation adaptée ---
def run_epoch(model, loader, optimizer=None):
    """
    Si optimizer est fourni → training
    Sinon → validation / test
    
    Adapté pour BCEWithLogitsLoss et multi-label accuracy.
    """
    if optimizer:
        model.train()
    else:
        model.eval()

    total_loss = 0
    total_correct_labels = 0 # Compteur pour l'accuracy par label
    total_samples = 0
    total_labels = 0         # Nombre total de labels

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        # Forward
        preds = model(xb)   # logits (B, num_labels)
        loss = criterion(preds, yb)

        # Entraînement seulement si optimizer est fourni
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Stats
        total_loss += loss.item() * xb.size(0)
        
        # --- CALCUL DE L'ACCURACY MULTI-LABEL ---
        # 1. Appliquer Sigmoid et seuil (typiquement 0.5) pour obtenir les prédictions binaires
        # preds > 0 est équivalent à torch.sigmoid(preds) > 0.5
        predicted_labels = (preds > 0).float()  # 0. ou 1. (B, num_labels)
        
        # 2. Comparer les prédictions binaires avec les labels (yb)
        # (predicted_labels == yb) donne un tenseur de booléens (True/False)
        # .sum() compte le nombre de labels correctement prédits (sur B * num_labels)
        total_correct_labels += (predicted_labels == yb).sum().item()
        
        total_samples += xb.size(0)
        total_labels += xb.size(0) * yb.size(1) # B * num_labels

    avg_loss = total_loss / total_samples
    # Accuracy par label = (labels corrects) / (labels totaux)
    accuracy = total_correct_labels / total_labels

    return avg_loss, accuracy

for epoch in range(10):



    train_loss, train_acc = run_epoch(model, train_loader, optimizer)
    val_loss, val_acc = run_epoch(model, val_loader)
    
    print(
        f"Epoch {epoch:02d}  "
        f"Train loss={train_loss:.4f}, acc={train_acc:.4f} |  "
        f"Val loss={val_loss:.4f}, acc={val_acc:.4f}"
    )

<>:6: SyntaxWarning: invalid escape sequence '\F'
<>:7: SyntaxWarning: invalid escape sequence '\F'
<>:8: SyntaxWarning: invalid escape sequence '\F'
<>:9: SyntaxWarning: invalid escape sequence '\F'
<>:10: SyntaxWarning: invalid escape sequence '\F'
<>:11: SyntaxWarning: invalid escape sequence '\F'
<>:6: SyntaxWarning: invalid escape sequence '\F'
<>:7: SyntaxWarning: invalid escape sequence '\F'
<>:8: SyntaxWarning: invalid escape sequence '\F'
<>:9: SyntaxWarning: invalid escape sequence '\F'
<>:10: SyntaxWarning: invalid escape sequence '\F'
<>:11: SyntaxWarning: invalid escape sequence '\F'
C:\Users\Vartkirl\AppData\Local\Temp\ipykernel_4300\3594237426.py:6: SyntaxWarning: invalid escape sequence '\F'
  X_train = np.load("D:\Formation_Data_Engineer\Data_FullStack\Data_Engineer_Full_Stack\Projet_groupe\X_train.npy")
C:\Users\Vartkirl\AppData\Local\Temp\ipykernel_4300\3594237426.py:7: SyntaxWarning: invalid escape sequence '\F'
  y_train = np.load("D:\Formation_Data_Engineer\Data_F

KeyboardInterrupt: 